In [1]:
import scipy.sparse as sp
import scipy.sparse.linalg as splinalg
import numpy as np

In [2]:
# 疎行列の例
size = 5  # 行列のサイズ
diagonals = [2, 3, 4]  # 対角成分
A = sp.diags(diagonals, offsets=[-1, 0, 1], shape=(size, size), format="csr")

In [3]:
# 固有値と固有ベクトルを計算
eigenvalues, eigenvectors = splinalg.eigsh(A, k=size - 1)

In [4]:
eigenvalues

array([-3.1427169 ,  4.14730805,  5.11408204,  9.18790708])

In [5]:
import numpy as np
from scipy.sparse import coo_matrix


class Mel1dSparse:
    def __init__(self, xs):
        self.size = len(xs)
        self.xs = xs
        self.xs_next = np.roll(xs, -1)
        self.hs = self.xs_next - xs
        self.hs = self.hs[:-1]

    def get_K(self, c=None):
        if c is None:
            c = np.ones(self.size - 1)

        row = []
        col = []
        data = []

        for i in range(self.size):
            if i == 0:
                row.extend([i, i])
                col.extend([i, i + 1])
                data.extend([-1 / self.hs[i] * c[i], 1 / self.hs[i] * c[i]])
            elif i == self.size - 1:
                row.extend([i, i])
                col.extend([i, i - 1])
                data.extend(
                    [-1 / self.hs[i - 1] * c[i - 1], 1 / self.hs[i - 1] * c[i - 1]]
                )
            else:
                row.extend([i, i, i])
                col.extend([i - 1, i, i + 1])
                data.extend(
                    [
                        1 / self.hs[i - 1] * c[i - 1],
                        -c[i - 1] / self.hs[i - 1] - c[i] / self.hs[i],
                        c[i] / self.hs[i],
                    ]
                )

        K_sparse = coo_matrix((data, (row, col)), shape=(self.size, self.size))
        return K_sparse

    def get_M(self, alpha=None):
        if alpha is None:
            alpha = np.ones(self.size - 1)

        row = []
        col = []
        data = []

        for i in range(self.size):
            if i == 0:
                row.extend([i, i])
                col.extend([i, i + 1])
                data.extend([self.hs[i] / 3 * alpha[i], self.hs[i] / 6 * alpha[i]])
            elif i == self.size - 1:
                row.extend([i, i])
                col.extend([i - 1, i])
                data.extend(
                    [
                        self.hs[i - 1] / 6 * alpha[i - 1],
                        self.hs[i - 1] / 3 * alpha[i - 1],
                    ]
                )
            else:
                row.extend([i, i, i])
                col.extend([i - 1, i, i + 1])
                data.extend(
                    [
                        self.hs[i] / 6 * alpha[i - 1],
                        self.hs[i] / 3 * (alpha[i - 1] + alpha[i]),
                        self.hs[i] / 6 * alpha[i],
                    ]
                )

        M_sparse = coo_matrix((data, (row, col)), shape=(self.size, self.size))
        return M_sparse

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import hbar, m_e, eV, epsilon_0, e, physical_constants
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh, eig
import importlib

In [9]:
xs = np.linspace(1e-20, 1e-9, 1000)
x_centers = (xs[1:] + xs[:-1]) / 2
v = -1 / x_centers
femeig = Mel1dSparse(xs)
K0 = femeig.get_K()
M = femeig.get_M()
K1 = femeig.get_M(v)

In [10]:
from scipy.sparse import csr_matrix

# まずcoo_matrixからcsr_matrixへ変換
K0_csr = csr_matrix(K0)
M_csr = csr_matrix(M)
K1_csr = csr_matrix(K1)

# csr_matrixを使用してスライスする
K0_dirichlet = K0_csr[1:-1, 1:-1]
M_dirichlet = M_csr[1:-1, 1:-1]
K1_dirichlet = K1_csr[1:-1, 1:-1]

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 2998 stored elements in COOrdinate format>

In [ ]:
K0_dirichlet = K0[1:-1, 1:-1]
M_dirichlet = M[1:-1, 1:-1]
K1_dirichlet = K1[1:-1, 1:-1]
k = 3
C0 = hbar**2 / (m_e)
C1 = e**2 / (4 * np.pi * epsilon_0)
val, vec = eigsh(
    -1 / 2 * C0 * K0_dirichlet + C1 * K1_dirichlet,
    k=k,
    M=M_dirichlet,
    which="LA",
    sigma=-20 * eV,
)